# 1. Импорт модулей.

In [ ]:
from google.colab import drive
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import *
from tensorflow.keras.models import Sequential, Model
import numpy as np
import pandas as pd
import os
import tensorflow.keras.backend as K
from sklearn.metrics import *
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.preprocessing import MinMaxScaler
import pickle
import seaborn as sns
from matplotlib import pyplot as plt 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model

from sklearn import metrics
from sklearn.metrics import classification_report

from keras.callbacks import Callback, TensorBoard

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# 2. Разделение классов аттак

Packet Length Variance      
Fwd IAT Std                 
Active Min                  
Active Mean                  
Active Max                    
Flow IAT Std                
Bwd IAT Mean                
PSH Flag Count                 
min_seg_size_forward         
Flow IAT Mean               
Active Std                  
Bwd IAT Min                  
Bwd IAT Max                   
Total Length of Fwd Packets    
Subflow Fwd Bytes            
Fwd PSH Flags                 
SYN Flag Count                
Fwd IAT Mean               
Bwd IAT Std                
Flow IAT Min                
Bwd Packet Length Min        
Init_Win_bytes_forward        
Fwd IAT Min        
Label

In [ ]:
file = pd.read_csv('/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/End/x_smt.csv')
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283771 entries, 0 to 283770
Data columns (total 24 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0    Packet Length Variance      283771 non-null  float64
 1    Active Min                  283771 non-null  int64  
 2    Fwd IAT Std                 283771 non-null  float64
 3   Active Mean                  283771 non-null  float64
 4    Active Max                  283771 non-null  int64  
 5    min_seg_size_forward        283771 non-null  int64  
 6    Flow IAT Std                283771 non-null  float64
 7    Bwd IAT Mean                283771 non-null  float64
 8    PSH Flag Count              283771 non-null  int64  
 9    Flow IAT Mean               283771 non-null  float64
 10  Total Length of Fwd Packets  283771 non-null  int64  
 11   Subflow Fwd Bytes           283771 non-null  int64  
 12   Bwd IAT Min                 283771 non-null  int64  
 13 

In [ ]:
data = []
target = []
for n, i in file.iterrows():
  a = []
  for j in i[:-1]:
    a.append(j)
  data.append(a)
  target.append(0 if i[-1] == 'BENIGN' else 1) #0 benign 1 attack
del file

In [ ]:
print('data example:')
print(data[:2])
print('target example:')
print(target[:2])

data example:
[[8424.2568359375, 50911, 5268490.0, 63678.19921875, 103175, 32, 4901981.5, 7518953.5, 1, 3541331.75, 322, 322, 637, 22252.53515625, 10300000, 0, 0, 6396442.0, 4645137.5, 47, 29200, 0, 234], [0.0, 0, 0.0, 0.0, 0, 32, 0.0, 0.0, 0, 57891.0, 0, 0, 0, 0.0, 0, 0, 0, 0.0, 0.0, 57891, 237, 0, 0]]
target example:
[1, 1]


Создание двух csv из target_df как набора 1 и 0 из верных маркировок и data_df как набора из признаков.

In [ ]:
#создание csv из таргет и даты
#target_df = pd.DataFrame(target)
#csv_target_df = target_df.to_csv('/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/smt_y_bin.csv')
#data_df = pd.DataFrame(data)
#csv_data_df = data_df.to_csv('/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/smt_x_bin.csv')

In [ ]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(np.nan_to_num(data).astype(float))
data = scaler.transform(np.nan_to_num(data).astype(float))
print(data.shape)
#data = data.reshape(data.shape[0], 23)

(283771, 23)


# 3. Решейпинг

In [ ]:
data = np.array(data)
target = np.array(target)
print(data.shape, target.shape)
print('----')
print(data[0].shape)
print('----')
print(data[:2])
print('----')


(283771, 23) (283771,)
----
(23,)
----
[[-0.99924106 -0.99881464 -0.8720885  -0.99851739 -0.99791566 -0.91655802
  -0.88029349 -0.87468411  1.         -0.93601997 -0.99977531 -0.99977531
  -0.99998938 -0.99936421 -0.82833333 -1.         -1.         -0.89339263
  -0.88656563 -0.99999892 -0.1088562  -1.         -0.9999961 ]
 [-1.         -1.         -1.         -1.         -1.         -0.91655802
  -1.         -1.         -1.         -0.99895387 -1.         -1.
  -1.         -1.         -1.         -1.         -1.         -1.
  -1.         -0.99895387 -0.99273682 -1.         -1.        ]]
----


In [ ]:
data = data.reshape(data.shape[0], data.shape[1], 1)
print(f"shape of data", data.shape)

shape of data (283771, 23, 1)


In [ ]:
from collections import Counter

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=1)
print(Counter(y_train))
print(Counter(y_test))

Counter({0: 113707, 1: 113309})
Counter({1: 28393, 0: 28362})


# X. Callbacks

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = '/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/End/Settings test ipynbs/lstm-5-13/.{epoch:02d}-{loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', save_freq=2)


In [ ]:
class PredictionCallback(tf.keras.callbacks.Callback):    
  def on_epoch_end(self, epoch, logs={}):
    print("prediction started")
    y_pred_calls1 = self.model.predict(X_test, batch_size=256, verbose = 2);
    y_pred_calls = np.argmax(y_pred_calls1, axis=1);
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_calls).ravel()
    sum = tn + fp + fn + tp
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1score = 2 * (precision * recall) / (precision + recall) 
    roc_auc = roc_auc_score(y_test, y_pred_calls)
    unique, counts = np.unique(y_pred_calls, return_counts=True)

    d = {'epoch': [epoch], 'tn': [tn], 'fp': [fp], 'fn': [fn], 'tp': [tp], 'sum':[sum], 'recall': [recall], 'precision':[precision], 'f1score':[f1score], 'rocauc':[roc_auc]}
    df = pd.DataFrame(data=d)
    df.to_csv('/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/End/Settings test ipynbs/lstm-5-13/ClasRepo-5-13.csv', mode='a', header = False, index = False)

In [ ]:
logdir = ("/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/End/Bin_logs/")
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

# 4. Моделирование бинарной нейросети LSTM.

In [ ]:
tf.keras.backend.clear_session()

#n_features = data_reshaped.shape[2]

lstm = Sequential()
lstm.add(Input((23, 1))) 
lstm.add(LSTM(23, recurrent_activation='sigmoid', return_sequences=True, recurrent_dropout=0))
for i in range(5):
  lstm.add(LSTM(13, recurrent_activation='sigmoid', return_sequences=True, recurrent_dropout=0))
lstm.add(Flatten())
lstm.add(Dense(2 ,activation = 'sigmoid'))
lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 23, 23)            2300      
                                                                 
 lstm_1 (LSTM)               (None, 23, 13)            1924      
                                                                 
 lstm_2 (LSTM)               (None, 23, 13)            1404      
                                                                 
 lstm_3 (LSTM)               (None, 23, 13)            1404      
                                                                 
 lstm_4 (LSTM)               (None, 23, 13)            1404      
                                                                 
 lstm_5 (LSTM)               (None, 23, 13)            1404      
                                                                 
 flatten (Flatten)           (None, 299)               0

Тренировка нейросети

In [ ]:
lstm.load_weights('/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/End/Settings test ipynbs/lstm-5-13/.240-0.03.hdf5')

In [ ]:
#lstm_history = lstm.fit(X_train, to_categorical(y_train), epochs=60, batch_size=256, verbose=2, callbacks=[PredictionCallback(), checkpoint])

In [ ]:
lstm_history = lstm.fit(X_train, to_categorical(y_train), epochs=60, batch_size=256, verbose=2)

Epoch 1/60
887/887 - 114s - loss: 0.1038 - accuracy: 0.9636 - 114s/epoch - 128ms/step
Epoch 2/60
887/887 - 95s - loss: 0.1036 - accuracy: 0.9642 - 95s/epoch - 107ms/step
Epoch 3/60


KeyboardInterrupt: ignored

In [ ]:
pickle.dump(lstm, open('/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/End/Settings test ipynbs/lstm-5-13/lstm_5hidden13neurons.pkl', "wb"))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......flatten
.........vars
......lstm
.........cell
............vars
...............0
...............1
...............2
.........vars
......lstm_1
.........cell
............vars
...............0
...............1
...............2
.........vars
......lstm_2
.........cell
............vars
...............0
...............1
...............2
.........vars
......lstm_3
.........cell
............vars
...............0
...............1
...............2
.........vars
......lstm_4
.........cell
............vars
...............0
...............1
...............2
.........vars
......lstm_5
.........cell
............vars
...............0
...............1
...............2
.........vars
...optimizer
......vars
.........0
...vars
Keras model archive saving:
File Name                                             Modified             Size
variables.h5                           

Получение данных от нейросети

In [ ]:
y_pred1 = lstm.predict(X_test, batch_size=256)
y_pred = np.argmax(y_pred1, axis=1)

222/222 [==============================] - 13s 47ms/step


In [ ]:
y_pred1

Вычисление метрик 

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
target_names = ['Benign', 'Attack']
print('==============================Classification Report=============================='.format(i))
print(classification_report(y_test, y_pred, target_names=target_names))
print('==============================Confussion Matrix and metrics=============================='.format(i))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
sum = tn + fp + fn + tp
print('Confussion Matrix:', "\n", confusion_matrix(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))
recall = tp / (tp + fn)
print('recall:', recall)
precision = tp / (tp + fp)
print('precision:', precision)
print('f1 score:', 2 * (precision * recall) / (precision + recall))
print('ROC AUC:', roc_auc_score(y_test, y_pred))
unique, counts = np.unique(y_pred, return_counts=True)
count_list = dict(zip(unique, counts))
print('Alghorythm marked', count_list[1], 'as attack and', count_list[0], "packets as benign")

==============================Classification Report==============================
              precision    recall  f1-score   support

      Benign       0.99      0.95      0.97     28362
      Attack       0.96      0.99      0.97     28393

    accuracy                           0.97     56755
   macro avg       0.97      0.97      0.97     56755
weighted avg       0.97      0.97      0.97     56755

==============================Confussion Matrix and metrics==============================
Confussion Matrix: 
 [[27069  1293]
 [  405 27988]]
Accuracy:  0.9700819311073914
recall: 0.9857359208255556
precision: 0.9558416720740411
f1 score: 0.9705586572805771
ROC AUC: 0.9700733761098372
Alghorythm marked 29281 as attack and 27474 packets as benign


In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir "/content/drive/MyDrive/CICIDS2017/2022. Подключение к СИАСТ/End/Bin_logs/"

Автотестирование (Оч долго)

In [ ]:
#import keras as K
#def Create_model(hidden):
    input = Input((23,1))
    x = input
    K.layers.LSTM(32, recurrent_activation='sigmoid', return_sequences=True, recurrent_dropout=0)(x)
    for i in range(hidden):
        x = K.layers.LSTM(32, recurrent_activation='sigmoid', return_sequences=True, recurrent_dropout=0)(x)
    x = K.layers.Flatten()(x)
    output = K.layers.Dense(2, activation = 'softmax')(x)
    return K.Model(inputs=input, outputs=output)

In [ ]:
#def report(y_pred):
  target_names = ['Benign', 'Attack']
  print('==============================Classification Report=============================='.format(i))
  print(classification_report(target, y_pred, target_names=target_names))
  print('==============================Confussion Matrix and metrics=============================='.format(i))
  tn, fp, fn, tp = confusion_matrix(target, y_pred).ravel()
  sum = tn + fp + fn + tp
  print('Confussion Matrix:', "\n", confusion_matrix(target, y_pred))
  print("Accuracy: ", accuracy_score(target, y_pred))
  recall = tp / (tp + fn)
  print('recall:', recall)
  precision = tp / (tp + fp)
  print('precision:', precision)
  print('f1 score:', 2 * (precision * recall) / (precision + recall))
  print('ROC AUC:', roc_auc_score(target, y_pred))
  unique, counts = np.unique(y_pred, return_counts=True)
  count_list = dict(zip(unique, counts))
  print('Alghorythm marked', count_list[1], 'as attack and', count_list[0], "packets as benign")

In [ ]:
#def Testing(hidden, epochs):
  for i in range(1, hidden, 1):  #Настройка кол-ва скрытых слоев
    Model = Create_model(hidden);
    Model.summary();
    Model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
    for z in range(0, 10, 5): #Настройка кол-ва эпох
      Model.fit(data, to_categorical(target), epochs=epochs, batch_size=256, verbose=1);
      y_pred1 = Model.predict(data, batch_size=256);
      y_pred = np.argmax(y_pred1, axis=1);
      report(y_pred);

In [ ]:
#Testing(2, 2)

In [ ]:
#tf.keras.backend.clear_session()